In [29]:
import pandas as pd
import requests
import re
import numpy as np
from datetime import datetime


In [30]:
conflict_df = pd.read_csv('datasets/ucdp-prio-acd-171.csv')
countries_codes = pd.read_csv('datasets/countries_codes_and_coordinates.csv')

In [31]:
conflict_df;


In [32]:
for index, row in conflict_df.iterrows():
    conflict_df.loc[index, 'location'] = re.sub(r'\([^)]*\)', '', conflict_df.loc[index, 'location'])
    conflict_df.loc[index, 'location'] = conflict_df.loc[index, 'location'].rstrip()
    conflict_df.loc[index, 'sidea'] = conflict_df.loc[index, 'sidea'].replace('Government of ', '')
    conflict_df.loc[index, 'startdate'] = datetime.strptime(conflict_df.loc[index, 'startdate'], '%Y-%m-%d').month
    #conflict_df.loc[index, 'ependdate'] = datetime.strptime(conflict_df.loc[index, 'ependdate'], '%Y-%m-%d').month

    if pd.isnull(row['terr']) :
        conflict_df.loc[index, 'terr'] = row['location']    

In [33]:
conflict_df = conflict_df.drop(['gwnoa2nd', 'gwnob2nd', 'gwnoloc', 'gwnoa', 
                  'gwnob', 'version', 'startdate2', 'startprec', 
                  'startdate2', 'startprec2' , 'ependprec', 'sidea2nd', 
                  'sideb2nd', 'epend', 'incomp'], axis=1)


In [34]:
conflict_df = conflict_df.rename(index=str, columns={ "startdate" : "start_month", "ependdate": "end_month"});


In [35]:
iso_countries = countries_codes['Country'].tolist()
iso_code = countries_codes['ISO2'].tolist()
sidea = conflict_df['sidea'].tolist()
location  = conflict_df['location'].tolist()

In [36]:
iso_conflict = []

for i in range(0, len(sidea)):
    if sidea[i] in iso_countries or location[i] in iso_countries:
        for j in range(0, len(iso_countries)): 
            if location[i] == iso_countries[j]:
                iso_conflict.append(iso_code[j])
                break
            elif sidea[i] == iso_countries[j]:
                iso_conflict.append(iso_code[j])
                break
    else:
        iso_conflict.append(0)


In [37]:
conflict_df = conflict_df.assign(ISO2 = iso_conflict )

In [38]:
conflict_df.to_csv('clean_conflict.csv')